### Hi everyone, in this notebook we will start with basic LGBMClassifier and then we will see how to optimize hyper paramters of LGBMClassifier with Optuna.

## Approach

1. Import libraries
2. Read the data
3. Check for missing values and target distribution
4. Create folds for Cross Validation
5. Fit with base LGBMClassifier
6. Tune hyperparamters with Optuna
7. Retrain the model with tuned hyperparameters.
8. Create submission files

## Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold # For creating folds
from sklearn.metrics import log_loss # Evaluation metrics

import optuna
#from optuna.integration.lightgbm import LGBMClassifier

## 2. Reading the train, test and sample submission file

In [ ]:
df = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
ss = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")

## 3. Basic data check

In [ ]:
print(f"Shape of train : {df.shape}")
print(f"Shape of test : {test.shape}")
print(f"Shape of sample submission : {ss.shape}")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
test.info()

There are no missing values in the both train and test datasets and all are integers, so the categories might be encoded already.

In [ ]:
sns.countplot(x= df.target)

Target column is imbalanced, so I will use StratifiedKFold for cross validation.

Since it is a baseline/starter model, I am not doing EDA and directly moving onto model building part.

## 4. Create folds for Cross Validation

Creating folds for the train dataset, so that we can train the model for the n folds, to avoid overfitting.

In [ ]:
df["kfold"] = -1
df = df.sample(frac=1,random_state=42).reset_index(drop=True)
y = df.target
kf = StratifiedKFold(n_splits=5)
for f, (t_,v_) in enumerate(kf.split(X=df,y=y)):
  df.loc[v_,"kfold"] = f

## 5. Basline model

In [ ]:
lgbm = LGBMClassifier(random_state=42)
logloss = []
lgbm_pred = 0
for f in range(5): # Looping around 5 folds
    
    #Splitting the data into train and validation set
    train = df[df.kfold!= f].reset_index(drop=True) 
    valid = df[df.kfold== f].reset_index(drop=True)
    
    #Creating X_train and y_train
    X_train = train.drop(["id","target", "kfold"], axis=1)
    y_train = train.target
    X_valid = valid.drop(["id","target", "kfold"], axis=1)
    y_valid = valid.target
    X_test = test.drop(["id"], axis=1)
    
    
    #Fitting the model
    lgbm.fit(X_train,y_train)
    
    #Predicting for valid and test datasets
    valid_preds = lgbm.predict_proba(X_valid)
    lgbm_pred += lgbm.predict_proba(X_test)/5
    
    #Calculating log loss
    logloss.append(log_loss(y_valid,valid_preds))
    
print(logloss)
print(sum(logloss)/len(logloss))

## 6. Optuna - Hyperparameter tuning

In [ ]:
def optimize(trial):
    param = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'objective': 'multiclass',
        #'metric' : ''
        "random_state" : 42}



    model = LGBMClassifier(**param)
    logloss = []
    for f in range(5):
        train = df[df.kfold!= f].reset_index(drop=True)
        valid = df[df.kfold== f].reset_index(drop=True)

        X_train = train.drop(["id","target", "kfold"], axis=1)
        y_train = train["target"]
        X_valid = valid.drop(["id","target", "kfold"], axis=1)
        y_valid = valid["target"]

        model.fit(X_train,y_train)
        pred = model.predict_proba(X_valid)
        fold_logloss = log_loss(y_valid, pred)
        logloss.append(fold_logloss)
    
    return np.mean(logloss)

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(optimize, n_trials=15)

The best parameters are below.

In [ ]:
print(study.best_params)

## 7. Tuned model

Retraining with the tuned parameters

In [ ]:
param = {
    'lambda_l1': 7.288238692320857, 
    'lambda_l2': 0.0011320378225972616, 
    'num_leaves': 23, 
    'feature_fraction': 0.8423047951534829, 
    'bagging_fraction': 0.8752188255110098, 
    'bagging_freq': 1, 
    'min_child_samples': 93,
    'objective': 'multiclass',
    'random_state' : 42
}
lgbm = LGBMClassifier(**param)
logloss = []
lgbm_pred = 0
for f in range(5): # Looping around 5 folds
    
    #Splitting the data into train and validation set
    train = df[df.kfold!= f].reset_index(drop=True) 
    valid = df[df.kfold== f].reset_index(drop=True)
    
    #Creating X_train and y_train
    X_train = train.drop(["id","target", "kfold"], axis=1)
    y_train = train.target
    X_valid = valid.drop(["id","target", "kfold"], axis=1)
    y_valid = valid.target
    X_test = test.drop(["id"], axis=1)
    
    
    #Fitting the model
    lgbm.fit(X_train,y_train)
    
    #Predicting for valid and test datasets
    valid_preds = lgbm.predict_proba(X_valid)
    lgbm_pred += lgbm.predict_proba(X_test)/5
    
    #Calculating log loss
    logloss.append(log_loss(y_valid,valid_preds))
    
print(logloss)
print(sum(logloss)/len(logloss))

In [ ]:
print(sum(logloss)/len(logloss))

## 8. Submission file

In [ ]:
ss["Class_1"] = lgbm_pred[:,0]
ss["Class_2"] = lgbm_pred[:,1]
ss["Class_3"] = lgbm_pred[:,2]
ss["Class_4"] = lgbm_pred[:,3]
ss["Class_5"] = lgbm_pred[:,4]
ss["Class_6"] = lgbm_pred[:,5]
ss["Class_7"] = lgbm_pred[:,6]
ss["Class_8"] = lgbm_pred[:,7]
ss["Class_9"] = lgbm_pred[:,8]
ss.to_csv("/kaggle/working/tuned_lgbm_sub.csv", index=False)

If you like the notebook kindly upvote it. It will motivate me to write more notebooks. :)

### Thank you!